### Створіть keyspace з найпростішої стратегією реплікації 

https://docs.datastax.com/en/dse/6.8/cql/cql/cql_using/useCreateKeyspace.html 


## I created keyspace in using cqlsh by the next QUERY

#### CREATE KEYSPACE IF NOT EXISTS first_keyspace
####    WITH REPLICATION = {
####        'class' : 'SimpleStrategy',
####        'replication_factor' : 1
####        };

In [10]:
# !pip install cassandra-driver

from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1'],port=9042)
session = cluster.connect('first_keyspace', wait_for_all_pools=True)
session.execute('USE first_keyspace')Створіть keyspace з найпростішої стратегією реплікації https://docs.datastax.com/en/dse/6.8/cql/cql/cql_using/useCreateKeyspace.html 



Аналогічно завданням по MongoDB Вам необхідно змоделювати інтернет-магазин який містить різноманітні товара (тобто у яких різний набір властивостей) - таблиця items. 
#### Необхідно, щоб пошук швидко працював для визначеної категорії товарів

Для набору властивостей товару виберіть базові характеристики однакові для всіх товарів (назва, ціна, виробник, ...), а для властивостей які відрізняються використовуйте тип map (з індексом для можливості пошуку по її вмісту)


In [15]:
# Create items table and indexes for attributes
query_for_items_table_creation='''
CREATE TABLE IF NOT EXISTS first_keyspace.items
(
    category_name text,
    price         float,
    item_name     text,
    item_id       int,
    manufacturer  text,
    attributes    map<text, text>,
    PRIMARY KEY (category_name, price, item_name)
);'''

print(session.execute(query_for_items_table_creation))

query_for_index_creation_attributes_keys = '''
CREATE INDEX IF NOT EXISTS ON first_keyspace.items (keys (attributes));
'''
print(session.execute(query_for_index_creation_attributes_keys))


query_for_index_creation_attributes_values = '''
CREATE INDEX IF NOT EXISTS ON first_keyspace.items (values (attributes));
'''
print(session.execute(query_for_index_creation_attributes_values))


# Insert values examples
queries_for_insert = [
'''INSERT INTO first_keyspace.items (item_name, price, manufacturer, category_name, attributes, item_id)
VALUES ('Iphone 15 ProMax', 850.00, 'Apple', 'Phones', {'country': 'China'}, 1);''',
'''INSERT INTO first_keyspace.items (item_name, price, manufacturer, category_name, attributes, item_id)
VALUES ('Iphone 15 Pro', 1150.00, 'Apple', 'Phones', {}, 2);''',
'''INSERT INTO first_keyspace.items (item_name, price, manufacturer, category_name, attributes, item_id)
VALUES ('Iphone 15', 1850.00, 'Apple', 'Phones', {'country': 'China'}, 3);''',

'''INSERT INTO first_keyspace.items (item_name, price, manufacturer, category_name, attributes, item_id)
VALUES ('TV 9823 LK PRO', 3850.00, 'Samsung', 'TV', {'color': 'white'}, 4);''',
'''INSERT INTO first_keyspace.items (item_name, price, manufacturer, category_name, attributes, item_id)
VALUES ('АОТУ 123 ОЩ', 4550.00, 'Sony', 'TV', {'color': 'black'}, 5);''',
'''INSERT INTO first_keyspace.items (item_name, price, manufacturer, category_name, attributes, item_id)
VALUES ('TV 487 HJH ', 2650.00, 'LG', 'TV', {}, 6);''']

for insert_query in queries_for_insert:
    print(session.execute(insert_query))

### 1. Напишіть запит, який показує структуру створеної таблиці (команда DESCRIBE)

In [29]:
for row in session.execute("DESCRIBE first_keyspace.items;"):
#     [print(x) for x in row.split('\n')]
    print('keyspace_name: ', row.keyspace_name)
    print('type: ', row.type) 
    print('name: ', row.name)
    print('create_statement: ', row.create_statement)

keyspace_name:  first_keyspace
type:  table
name:  items
create_statement:  CREATE TABLE first_keyspace.items (
    category_name text,
    price float,
    item_name text,
    item_id int,
    manufacturer text,
    attributes map<text, text>,
    PRIMARY KEY (category_name, price, item_name)
) WITH CLUSTERING ORDER BY (price ASC, item_name ASC)
    AND additional_write_policy = '99p'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND cdc = false
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND memtable = 'default'
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND extensions = {}
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_

### 2. Напишіть запит, який виводить усі товари в певній категорії відсортовані за ціною

In [34]:
query_to_find_special_category = '''
SELECT *
FROM first_keyspace.items
WHERE category_name = 'TV'
ORDER BY price DESC;
'''

for row in session.execute(query_to_find_special_category):
    print(row)

Row(category_name='TV', price=4550.0, item_name='АОТУ 123 ОЩ', attributes=OrderedMapSerializedKey([('color', 'black')]), item_id=5, manufacturer='Sony')
Row(category_name='TV', price=3850.0, item_name='TV 9823 LK PRO', attributes=OrderedMapSerializedKey([('color', 'white')]), item_id=4, manufacturer='Samsung')
Row(category_name='TV', price=2650.0, item_name='TV 487 HJH ', attributes=None, item_id=6, manufacturer='LG')


### 3. Напишіть запити, які вибирають товари за різними критеріями в межах певної категорії (тут де треба замість індексу використайте Matirialized view):
- назва, 
- ціна (в проміжку), 
- ціна та виробник 


In [38]:
query_find_by_price_range = '''
SELECT *
FROM first_keyspace.items
WHERE category_name = 'TV'
  AND price > 2000
  AND price < 3000;
'''

print('Find the items by PRICE RANGE')
for row in session.execute(query_find_by_price_range):
    print(row)

Find the items by PRICE RANGE
Row(category_name='TV', price=2650.0, item_name='TV 487 HJH ', attributes=None, item_id=6, manufacturer='LG')


In [39]:
# Changed in cassandra.yaml parameter enable_materialized_views: true

query_for_mv_for_find_name_creation = '''CREATE MATERIALIZED VIEW IF NOT EXISTS first_keyspace.items_name_mv AS
SELECT *
FROM first_keyspace.items
WHERE category_name IS NOT NULL
  AND price IS NOT NULL
  AND item_name IS NOT NULL
  AND manufacturer IS NOT NULL
PRIMARY KEY (category_name, item_name, manufacturer, price);
'''
print(session.execute(query_for_mv_for_find_name_creation))


query_find_by_name = '''
SELECT *
FROM first_keyspace.items_name_mv
WHERE category_name = 'Phones'
  AND item_name = 'Iphone 15';
'''

print('Find the items by NAME')
for row in session.execute(query_find_by_name):
    print(row)

Find the items by NAME
Row(category_name='Phones', item_name='Iphone 15', manufacturer='Apple', price=1850.0, attributes=OrderedMapSerializedKey([('country', 'China')]), item_id=3)


In [40]:
# Changed in cassandra.yaml parameter enable_materialized_views: true

query_for_mv_for_find_price_and_manufactor_creation = '''
CREATE MATERIALIZED VIEW IF NOT EXISTS first_keyspace.items_manufactor_and_price_mv AS
SELECT *
FROM first_keyspace.items
WHERE category_name IS NOT NULL
  AND price IS NOT NULL
  AND item_name IS NOT NULL
  AND manufacturer IS NOT NULL
PRIMARY KEY (category_name, manufacturer, price, item_name);
'''
print(session.execute(query_for_mv_for_find_price_and_manufactor_creation))


query_find_by_price_and_manufactor = '''
SELECT *
FROM first_keyspace.items_manufactor_and_price_mv
WHERE category_name = 'Phones'
  AND manufacturer = 'Apple'
  AND price > 1050;
'''

print('Find the items by PRICE AND MANUFACTOR')
for row in session.execute(query_find_by_price_and_manufactor):
    print(row)

Find the items by PRICE AND MANUFACTOR
Row(category_name='Phones', manufacturer='Apple', price=1150.0, item_name='Iphone 15 Pro', attributes=None, item_id=2)
Row(category_name='Phones', manufacturer='Apple', price=1850.0, item_name='Iphone 15', attributes=OrderedMapSerializedKey([('country', 'China')]), item_id=3)


### 4. Напишіть запити, які вибирають товари за:
- наявність певних характеристик
- певна характеристика та її значення


In [41]:
query_find_by_attributes_keys = '''
SELECT *
FROM first_keyspace.items
WHERE attributes CONTAINS KEY 'color';
'''

for row in session.execute(query_find_by_attributes_keys):
    print(row)

Row(category_name='TV', price=3850.0, item_name='TV 9823 LK PRO', attributes=OrderedMapSerializedKey([('color', 'white')]), item_id=4, manufacturer='Samsung')
Row(category_name='TV', price=4550.0, item_name='АОТУ 123 ОЩ', attributes=OrderedMapSerializedKey([('color', 'black')]), item_id=5, manufacturer='Sony')


In [43]:
query_find_by_attributes_values = '''
SELECT *
FROM first_keyspace.items
WHERE attributes CONTAINS 'white';
'''

for row in session.execute(query_find_by_attributes_values):
    print(row)

Row(category_name='TV', price=3850.0, item_name='TV 9823 LK PRO', attributes=OrderedMapSerializedKey([('color', 'white')]), item_id=4, manufacturer='Samsung')


### 5. Оновити опис товару:
- змінить існуючі значення певної характеристики 
- додайте нові властивості (характеристики) товару
- видалить характеристику товару


In [50]:
query_change_existing_value = '''
UPDATE first_keyspace.items
SET attributes['country'] = 'USA'
WHERE category_name = 'Phones'
  AND price = 1150
  AND item_name = 'Iphone 15 Pro';
'''

print(session.execute(query_change_existing_value))

query_after_update = '''
SELECT attributes 
FROM first_keyspace.items 
WHERE category_name = 'Phones'
  AND price = 1150
  AND item_name = 'Iphone 15 Pro';
  '''

for row in session.execute(query_after_update):
    print(row)

Row(attributes=OrderedMapSerializedKey([('country', 'USA')]))


In [51]:
query_add_new_attributes = '''
UPDATE first_keyspace.items
SET attributes['description'] = 'Garanty for a 1 year'
WHERE category_name = 'Phones'
  AND price = 850
  AND item_name = 'Iphone 15 ProMax';
'''

print(session.execute(query_add_new_attributes))

query_after_update = '''
SELECT attributes 
FROM first_keyspace.items 
WHERE category_name = 'Phones'
  AND price = 850
  AND item_name = 'Iphone 15 ProMax';
  '''

for row in session.execute(query_after_update):
    print(row)

Row(attributes=OrderedMapSerializedKey([('country', 'China'), ('description', 'Garanty for a 1 year')]))


In [52]:
query_delete_attributes = '''
DELETE attributes['color']
FROM first_keyspace.items
WHERE category_name = 'TV'
  AND price = 3850
  AND item_name = 'TV 9823 LK PRO';
'''

print(session.execute(query_delete_attributes))

query_after_update = '''
SELECT attributes 
FROM first_keyspace.items 
WHERE category_name = 'TV'
  AND price = 3850
  AND item_name = 'TV 9823 LK PRO';
  '''

for row in session.execute(query_after_update):
    print(row)

Row(attributes=None)


####  Створіть таблицю orders в якій міститься ім'я замовника і інформація про замовлення: перелік id-товарів у замовленні, вартість замовлення, дата замовлення, .... 

### Для кожного замовника повинна бути можливість швидко шукати його замовлення і виконувати по них запити.


In [53]:
# Create items table and indexes for attributes
query_for_orders_table_creation='''
CREATE TABLE IF NOT EXISTS first_keyspace.orders
(
    order_id           bigint,
    customer_id        bigint,
    total_sum          float,
    order_created_date date,
    item_ids           list<int>,
    PRIMARY KEY (customer_id, order_created_date, order_id)
);
'''

print(session.execute(query_for_orders_table_creation))


# Insert values examples
queries_for_insert = [
'''INSERT INTO first_keyspace.orders (order_id, customer_id, total_sum, order_created_date, item_ids)
VALUES (1, 1, 850.00, toDate('2024-01-01'), [1, 2, 3]);''',
'''INSERT INTO first_keyspace.orders (order_id, customer_id, total_sum, order_created_date, item_ids)
VALUES (2, 2, 1150.00, toDate('2024-01-02'), [4, 2, 3]);''',
'''INSERT INTO first_keyspace.orders (order_id, customer_id, total_sum, order_created_date, item_ids)
VALUES (3, 3, 1850.00, toDate('2024-01-03'), [5, 4, 3]);''',

'''INSERT INTO first_keyspace.orders (order_id, customer_id, total_sum, order_created_date, item_ids)
VALUES (4, 1, 3850.00, toDate('2024-01-04'), [1, 2, 4]);''',
'''INSERT INTO first_keyspace.orders (order_id, customer_id, total_sum, order_created_date, item_ids)
VALUES (5, 1, 4550.00, toDate('2024-01-05'), [5, 1, 3]);''']

for insert_query in queries_for_insert:
    print(session.execute(insert_query))

### 1.Напишіть запит, який показує структуру створеної таблиці (команда DESCRIBE)

In [54]:
for row in session.execute("DESCRIBE first_keyspace.orders;"):
#     [print(x) for x in row.split('\n')]
    print('keyspace_name: ', row.keyspace_name)
    print('type: ', row.type) 
    print('name: ', row.name)
    print('create_statement: ', row.create_statement)

keyspace_name:  first_keyspace
type:  table
name:  orders
create_statement:  CREATE TABLE first_keyspace.orders (
    customer_id bigint,
    order_created_date date,
    order_id bigint,
    total_sum float,
    item_ids list<int>,
    PRIMARY KEY (customer_id, order_created_date, order_id)
) WITH CLUSTERING ORDER BY (order_created_date ASC, order_id ASC)
    AND additional_write_policy = '99p'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND cdc = false
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND memtable = 'default'
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND extensions = {}
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flu

### 2. Для замовника виведіть всі його замовлення відсортовані за часом коли вони були зроблені

In [55]:
query_orders_by_customer = '''
SELECT *
FROM first_keyspace.orders
WHERE customer_id = 1
ORDER BY order_created_date DESC;
'''

for row in session.execute(query_orders_by_customer):
    print(row)

Row(customer_id=1, order_created_date=Date(19727), order_id=5, item_ids=[5, 1, 3], total_sum=4550.0)
Row(customer_id=1, order_created_date=Date(19726), order_id=4, item_ids=[1, 2, 4], total_sum=3850.0)
Row(customer_id=1, order_created_date=Date(19723), order_id=1, item_ids=[1, 2, 3], total_sum=850.0)


### 3. Для замовника знайдіть замовлення з певним товаром

In [58]:
query_create_index_for_items = '''
CREATE INDEX IF NOT EXISTS ON first_keyspace.orders (item_ids);
'''
print(session.execute(query_create_index_for_items))

query_orders_by_item = '''
SELECT *
FROM first_keyspace.orders
WHERE customer_id = 1
  AND item_ids CONTAINS 2;

'''

for row in session.execute(query_orders_by_item):
    print(row)

Row(customer_id=1, order_created_date=Date(19723), order_id=1, item_ids=[1, 2, 3], total_sum=850.0)
Row(customer_id=1, order_created_date=Date(19726), order_id=4, item_ids=[1, 2, 4], total_sum=3850.0)


### 4. Для замовника знайдіть замовлення за певний період часу і їх кількість

In [60]:
query_amount_of_orders_by_one_customer = '''
SELECT count(order_id)
FROM first_keyspace.orders
WHERE customer_id = 1
  AND order_created_date >= '2024-01-02';

'''

for row in session.execute(query_amount_of_orders_by_one_customer):
    print(row)

Row(system_count_order_id=2)


### 5. Для кожного замовників визначте суму на яку були зроблені усі його замовлення

In [61]:
query_total_sum_by_each_customer = '''
SELECT customer_id, sum(total_sum)
FROM first_keyspace.orders
GROUP BY customer_id;

'''

for row in session.execute(query_total_sum_by_each_customer):
    print(row)

Row(customer_id=2, system_sum_total_sum=1150.0)
Row(customer_id=3, system_sum_total_sum=1850.0)
Row(customer_id=1, system_sum_total_sum=9250.0)


### 6. Для кожного замовників визначте замовлення з максимальною вартістю

In [62]:
query_total_max_by_each_customer = '''
SELECT customer_id, max(total_sum)
FROM first_keyspace.orders
GROUP BY customer_id;

'''

for row in session.execute(query_total_max_by_each_customer):
    print(row)

Row(customer_id=2, system_max_total_sum=1150.0)
Row(customer_id=3, system_max_total_sum=1850.0)
Row(customer_id=1, system_max_total_sum=4550.0)


### 7. Модифікуйте певне замовлення додавши / видаливши один або кілька товарів при цьому також змінюючи вартість замовлення

In [63]:
query_items_adding_and_total_update = '''
UPDATE first_keyspace.orders
SET item_ids = item_ids + [6], total_sum = 1300
WHERE customer_id = 1
  AND order_created_date = '2024-01-01'
  AND order_id = 1;
'''
print(session.execute(query_items_adding_and_total_update))

query_after_update = '''
SELECT * 
FROM first_keyspace.orders
WHERE customer_id = 1
  AND order_created_date = '2024-01-01'
  AND order_id = 1;
'''

for row in session.execute(query_after_update):
    print(row)

Row(customer_id=1, order_created_date=Date(19723), order_id=1, item_ids=[1, 2, 3, 6], total_sum=1300.0)


### 8. Для кожного замовлення виведіть час коли його ціна були занесена в базу (SELECT WRITETIME)

In [80]:
from datetime import datetime

query_for_mv_by_orders = '''
CREATE MATERIALIZED VIEW IF NOT EXISTS first_keyspace.orders_by_order AS
SELECT *
FROM first_keyspace.orders
WHERE order_id IS NOT NULL
--   AND total_sum IS NOT NULL
  AND order_created_date IS NOT NULL
  AND customer_id IS NOT NULL
PRIMARY KEY (order_id, order_created_date, customer_id);
'''
print(session.execute(query_for_mv_by_orders))


query_total_sum_time_update = '''
SELECT order_id, WRITETIME(total_sum)
FROM first_keyspace.orders_by_order
GROUP BY order_id;
'''

for row in session.execute(query_total_sum_time_update):
    print('\norder_id: ', row.order_id)
    print('update time for total_sum: ', datetime.fromtimestamp(row.writetime_total_sum / 1e6))


order_id:  2
update time for total_sum:  2024-03-13 22:13:09.943385

order_id:  3
update time for total_sum:  2024-03-13 22:13:09.952518

order_id:  4
update time for total_sum:  2024-03-13 22:13:09.968639

order_id:  5
update time for total_sum:  2024-03-13 22:13:09.984826

order_id:  1
update time for total_sum:  2024-03-13 22:32:41.534155


### 9. Створіть замовлення з певним часом життя (TTL), після якого воно видалиться 

In [85]:
import time

query_order_with_TTL_for_10_sec = '''
INSERT INTO first_keyspace.orders (order_id, customer_id, total_sum, order_created_date, item_ids)
VALUES (6, 2, 8850.00, toDate('2024-01-08'), [5, 1, 3, 4, 2])
    USING TTL 10;

'''
print(session.execute(query_order_with_TTL_for_10_sec))

query_check = ''' 
SELECT * 
FROM first_keyspace.orders 
WHERE customer_id = 2 
AND order_created_date = '2024-01-08' 
AND order_id = 6'''


print('Check that it was created')
for row in session.execute(query_check):
    print(row)
    
print('Wait 10 second and check if it is deleted')
time.sleep(11)
for row in session.execute(query_check):
    print(row)

print('The end')

Check that it was created
Row(customer_id=2, order_created_date=Date(19730), order_id=6, item_ids=[5, 1, 3, 4, 2], total_sum=8850.0)
Wait 10 second and check if it is deleted
The end


### 10. Поверніть замовлення у форматі JSON

In [87]:
query_get_JSON = '''
SELECT JSON *
FROM first_keyspace.orders;

'''

for row in session.execute(query_get_JSON):
    print(row.json)

{"customer_id": 2, "order_created_date": "2024-01-02", "order_id": 2, "item_ids": [4, 2, 3], "total_sum": 1150.0}
{"customer_id": 3, "order_created_date": "2024-01-03", "order_id": 3, "item_ids": [5, 4, 3], "total_sum": 1850.0}
{"customer_id": 1, "order_created_date": "2024-01-01", "order_id": 1, "item_ids": [1, 2, 3, 6], "total_sum": 1300.0}
{"customer_id": 1, "order_created_date": "2024-01-04", "order_id": 4, "item_ids": [1, 2, 4], "total_sum": 3850.0}
{"customer_id": 1, "order_created_date": "2024-01-05", "order_id": 5, "item_ids": [5, 1, 3], "total_sum": 4550.0}


### 11. Додайте замовлення у форматі JSON

In [88]:
query_insert_with_JSON = '''
INSERT INTO first_keyspace.orders
    JSON '{
  "order_id": 7,
  "customer_id": 3,
  "total_sum": 5040,
  "order_created_date": "2024-02-10",
  "item_ids": [5, 3, 1, 2]
}';

'''
print(session.execute(query_insert_with_JSON))


query_check = ''' 
SELECT * 
FROM first_keyspace.orders 
WHERE customer_id = 3
AND order_created_date = '2024-02-10' 
AND order_id = 7'''

for row in session.execute(query_check):
    print(row)

Row(customer_id=3, order_created_date=Date(19763), order_id=7, item_ids=[5, 3, 1, 2], total_sum=5040.0)
